<a href="https://colab.research.google.com/github/deepak-kannan7/ChatPDF/blob/main/ChatPDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%writefile app.py
import streamlit as st
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import PyPDFLoader
from langchain.chains import SimpleSequentialChain
import os
import PyPDF2
from io import BytesIO

os.environ["OPENAI_API_KEY"] = ''

# Set the title and icon for the browser tab
st.set_page_config(page_title="File Summary", page_icon="🚀")

def generate_summary(uploaded_file):
    prompt_template_identification = """Identify all the information in the following:
    "{text}"
    IDENTIFIED INFORMATION:"""

    prompt1 = PromptTemplate.from_template(prompt_template_identification)

    prompt_template_summary = """Explain the following content in about 500 words:
    "{text}"
    EXPLANATION:"""

    prompt1 = PromptTemplate.from_template(prompt_template_identification)
    prompt2 = PromptTemplate.from_template(prompt_template_summary)

    # Define LLM chain
    llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-16k")
    chain1 = LLMChain(llm=llm, prompt=prompt1)
    chain2 = LLMChain(llm=llm,prompt=prompt2)

    # Define StuffDocumentsChain
    # stuff_chain = StuffDocumentsChain(
    #     llm_chain=llm_chain, document_variable_name="text"
    # )

    overall_chain = SimpleSequentialChain(chains=[chain1,chain2])

    pdf_content = uploaded_file.read()

    # Create a BytesIO object from the content
    pdf_stream = BytesIO(pdf_content)

    # Create a PDF reader object using the content
    pdf_reader = PyPDF2.PdfReader(pdf_stream)

    # Loop through each page and extract text content
    all_text=""
    for page in pdf_reader.pages:

        page_text = page.extract_text()
        all_text += page_text

    # loader = PyPDFLoader(pdf_path)
    # docs = pdf_stream.extractText()
    return overall_chain.run(all_text)

def main():
    st.title("PDF Summary Generator")

    uploaded_file = st.file_uploader("Upload a PDF file", type=["pdf"])

    generate_button = st.button("Generate")

    if generate_button:

        if uploaded_file is not None:
            st.write("File uploaded!")

            # Display a loading spinner while generating summary
            with st.spinner("Generating summary..."):
                summary = generate_summary(uploaded_file)

            # Show the generated summary
            st.subheader("Summary:")
            st.write(summary)

        else:
            st.write("Please upload a file.")

if __name__ == "__main__":
    main()